In [2]:
from langchain.schema import SystemMessage
import streamlit as st
import os
import requests
from typing import Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import DuckDuckGoSearchAPIWrapper
from langchain_community.utilities import WikipediaAPIWrapper

#     """
#    (EN)
# In a new Jupyter notebook create a research AI agent and give it custom tools.
# The agent should be able to do the following tasks:
# Search in Wikipedia
# Search in DuckDuckGo
# Scrape and extract the text of any website.
# Save the research to a .txt file
# Run the agent with this query: "Research about the XZ backdoor", the agent should try to search in Wikipedia or DuckDuckGo, if it finds a website in DuckDuckGo it should enter the website and extract it's content, then it should finish by saving the research to a .txt file.
# (KR)
# 새로운 Jupyter notebook에서 리서치 AI 에이전트를 만들고 커스텀 도구를 부여합니다.
# 에이전트는 다음 작업을 수행할 수 있어야 합니다:
# Wikipedia에서 검색
# DuckDuckGo에서 검색
# 웹사이트의 텍스트를 스크랩하고 추출합니다.
# 리서치 결과를 .txt 파일에 저장하기
# 다음 쿼리로 에이전트를 실행합니다: "Research about the XZ backdoor" 라는 쿼리로 에이전트를 실행하면, 에이전트는 Wikipedia 또는 DuckDuckGo에서 검색을 시도하고, DuckDuckGo에서 웹사이트를 찾으면 해당 웹사이트에 들어가서 콘텐츠를 추출한 다음 .txt 파일에 조사 내용을 저장하는 것으로 완료해야 합니다.

# """


llm = ChatOpenAI(temperature=0.1, model_name="gpt-4o-mini")

#WikiPedia
class WikipediaResearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="The query you will search for.Example query: Research about the XZ backdoor"
    )

class WikipediaResearchTool(BaseTool):
    name:str  = "WikipediaResearchTool"
    description:str = """
    Use this tool to search the desired topic.
    It takes a query as an argument.
    
    """
    args_schema: Type[
        WikipediaResearchToolArgsSchema
    ] = WikipediaResearchToolArgsSchema

    def _run(self, query):
        ddg = WikipediaAPIWrapper()
        return ddg.run(query)

#DuckDuckGo
class DuckDuckGoResearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="The query you will search for.Example query: Research about the XZ backdoor"
    )

class DuckDuckGoResearchTool(BaseTool):
    name:str  = "DuckDuckGoResearchTool"
    description:str = """
    Use this tool to search the desired topic.
    It takes a query as an argument.
    
    """
    args_schema: Type[
        DuckDuckGoResearchToolArgsSchema
    ] = DuckDuckGoResearchToolArgsSchema

    def _run(self, query):
        ddg = DuckDuckGoSearchAPIWrapper()
        return ddg.run(query)


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        WikipediaResearchTool(),
        DuckDuckGoResearchTool()
    ],
    #agent keyword arguments
    agent_kwargs={
        "system_message": SystemMessage(
            content="""
            You are a research AI agent.
        """
        )
    },
)


result = agent.invoke("Research about the XZ backdoor.")

#save as a txt file

f = open("research_result.txt", "w")
f.write(str(result['output']))
f.close()




> Entering new AgentExecutor chain...

Invoking: `WikipediaResearchTool` with `{'query': 'XZ backdoor'}`


Page: XZ Utils backdoor
Summary: In February 2024, a malicious backdoor was introduced to the Linux build of the xz utility within the liblzma library in versions 5.6.0 and 5.6.1 by an account using the name "Jia Tan". The backdoor gives an attacker who possesses a specific Ed448 private key remote code execution capabilities on the affected Linux system. The issue has been given the Common Vulnerabilities and Exposures number CVE-2024-3094 and has been assigned a CVSS score of 10.0, the highest possible score.
While xz is commonly present in most Linux distributions, at the time of discovery the backdoored version had not yet been widely deployed to production systems, but was present in development versions of major distributions. The backdoor was discovered by the software developer Andres Freund, who announced his findings on 29 March 2024.

Page: XZ Utils
Summary: XZ Utils 